This is a basic script to monitor the lance_db updates.

In [1]:
import polars as pl
import lancedb
import time

pl.Config.set_fmt_str_lengths(200)
pl.Config.set_fmt_float("full")

/home/evan/Documents/degen_tracker/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


polars.config.Config

In [2]:
cd ../

/home/evan/Documents/degen_tracker


/home/evan/Documents/degen_tracker/.venv/lib/python3.12/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [6]:
# open lancedb table
uri: str = "logs"
db: lancedb.DBConnection = lancedb.connect(uri)
logs_tbl = db.open_table("logs")

# convert dataset to table (full in memory)
table = logs_tbl.to_pandas()
pl_df = pl.from_pandas(table)

In [10]:
pl_df.sort(by='block_number', descending=True).group_by('block_number').agg(pl.len().alias('transfers_count')).sort(by='block_number', descending=True).head(10)

block_number,transfers_count
i64,u32
13126668,84
13126667,86
13126666,84
13126665,37
13126664,76
13126663,63
13126662,93
13126661,106
13126660,237


In [7]:
pl_df.sort(by='block_number', descending=True).filter(pl.col('value_transferred') != 0).group_by('from').agg(pl.len().alias('address_swap_count')).sort(by='address_swap_count', descending=True).head(10)

from,address_swap_count
str,u32
"""0xe07f424a17f4489deee9fdada364a370b9290959""",25570
"""0x5c90ef4a8d0dad23fddb9483a420f3ae29078c74""",10174
"""0x19b43787fbc106d427be3ca3307d4fb0a73d83dc""",6120
"""0x40725d97f5352dfbf749de29ed328ac6189e51a9""",5411
"""0x7beca9348b79e545d5f0b8fd178dced98b4d45b2""",3534
"""0xbbb5ac2c0998d38e5f34dbfd9d6201a30993e555""",3458
"""0xd056c0eee354b24fe7c5d4ee762c4d7574badac1""",3404
"""0x4390bb890a3e87dbd62cc9a5738ac1ecbeafaca7""",3185
"""0xc647244fdba1ea6309ec332d65f3f8f6132b064b""",3145


In [5]:
while True:
    # open lancedb table
    uri: str = "logs"
    db: lancedb.DBConnection = lancedb.connect(uri)
    logs_tbl = db.open_table("logs")

    # convert dataset to table (full in memory)
    table = logs_tbl.to_pandas()
    pl_df = pl.from_pandas(table)

    pl_df.sort(by='block_number', descending=True)
    print(pl_df.shape)
    time.sleep(5)

(547201, 6)
(547201, 6)
(547591, 6)
(548181, 6)


KeyboardInterrupt: 